<a href="https://colab.research.google.com/github/alyssonmazoni/redalyc/blob/main/Match_Redalyc_OpenAlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
folder = '/content/drive/My Drive/Data infrastructure/Redalyc/'

Mounted at /content/drive/


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
import pandas as pd
from google.cloud import bigquery
import numpy as np
import pandas_gbq
import glob
import matplotlib.pyplot as plt
import json

# XML - JATS files

In [ ]:
jats_folder = 'Redalyc_jats_xml/'

## Upload JSON (converted from XML)

In [ ]:
journals = glob.glob(folder+jats_folder+'/*')

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.works`;

Query is running:   0%|          |

""


In [ ]:
for j in journals[-200:]:
  files = glob.glob(j+'/*/*.json')
  ids = []
  entries = []
  for fi in files:
    id = fi.split('/')[-1].split('.')[0]
    with open(fi) as filename:

      dict_entry = filename.read()
      #entries += [json.dumps(dict_entry, ensure_ascii=False)]
      entries += [dict_entry]
      ids += [id]

  J = pd.DataFrame({'id':ids,'work':entries})
  J.to_gbq(project_id = 'insyspo',
            destination_table = 'publicdb_redalyc_2024_02.works',
            if_exists = 'append')

100%|██████████| 1/1 [00:00<00:00, 1567.96it/s]


In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.works_json`;
CREATE TABLE `insyspo.publicdb_redalyc_2024_02.works_json` AS (
  SELECT id, ANY_VALUE(work) AS work
  FROM `insyspo.publicdb_redalyc_2024_02.works`
  GROUP BY id
);

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.works`;

Query is running:   0%|          |

""


# Metadata from JATS files

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.match_jats`;
CREATE TABLE `insyspo.publicdb_redalyc_2024_02.match_jats` AS (
  SELECT
      id,
      TRIM(COALESCE(
          JSON_EXTRACT(work,
            "$.article.front.article-meta.title-group.article-title['#text']"),
          JSON_EXTRACT(work,
            "$.article.front.article-meta.title-group.article-title")
      ),'"') AS title,
      TRIM(COALESCE(
        JSON_EXTRACT(work,
          "$.article.front.journal-meta.journal-title-group.journal-title['#text']"),
        JSON_EXTRACT(work,
         "$.article.front.journal-meta.journal-title-group.journal-title")
      ),'"') AS journal_title,
      TRIM(JSON_EXTRACT(work,
        "$.article.front.journal-meta.publisher.publisher-name"),
        '"') AS publisher,
      CONCAT(COALESCE(
          TRIM(JSON_EXTRACT(work,
            "$.article.front.article-meta.contrib-group.contrib[0].name.surname"),
            '"'),
          TRIM(JSON_EXTRACT(work,
            "$.article.front.article-meta.contrib-group.contrib.name.surname"),
          '"')
      ),' ',
      COALESCE(
          TRIM(JSON_EXTRACT(work,
            "$.article.front.article-meta.contrib-group.contrib[0].name.given-names"),
            '"'),
          TRIM(JSON_EXTRACT(work,
            "$.article.front.article-meta.contrib-group.contrib.name.given-names"),
          '"')
      )) AS first_author_name,
      CAST(TRIM(JSON_EXTRACT(work,
        "$.article.front.article-meta.pub-date.year"),
        '"') AS INT64) AS pub_year,
      TRIM(COALESCE(
        IF(JSON_EXTRACT(work,"$.article.front.journal-meta.issn['@pub-type']")='"ppub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.journal-meta.issn")[0],"$['@pub-type']")='"ppub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn[0]['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.journal-meta.issn")[1],"$['@pub-type']")='"ppub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn[1]['#text']"),NULL)
      ),'"') AS issn,
      TRIM(COALESCE(
        IF(JSON_EXTRACT(work,"$.article.front.journal-meta.issn['@pub-type']")='"epub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.journal-meta.issn")[0],"$['@pub-type']")='"epub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn[0]['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.journal-meta.issn")[1],"$['@pub-type']")='"epub"',
          JSON_EXTRACT(work,"$.article.front.journal-meta.issn[1]['#text']"),NULL)
      ),'"') AS eissn,
      TRIM(COALESCE(
        IF(JSON_EXTRACT(work,"$.article.front.article-meta.article-id['@pub-id-type']")='"doi"',
          JSON_EXTRACT(work,"$.article.front.article-meta.article-id['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.article-meta.article-id")[0],"$['@pub-id-type']")='"doi"',
          JSON_EXTRACT(work,"$.article.front.article-meta.article-id[0]['#text']"),NULL),
        IF(JSON_EXTRACT(JSON_EXTRACT_ARRAY(work,"$.article.front.article-meta.article-id")[1],"$['@pub-id-type']")='"doi"',
          JSON_EXTRACT(work,"$.article.front.article-meta.article-id[1]['#text']"),NULL)
      ),'"') AS doi,
  FROM `insyspo.publicdb_redalyc_2024_02.works_json`
);

Query is running:   0%|          |

""


# Metadata from PDFs

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.match_pdfs`;
CREATE TABLE `insyspo.publicdb_redalyc_2024_02.match_pdfs` AS (
  SELECT CAST(recordIdentifier AS STRING) AS id,
  dc_title AS title,
  dc_rights AS journal_title,
  dc_publisher AS publisher,
  SPLIT(dc_creator,'|')[0] AS first_author_name,
  dc_date AS pub_year,
  REGEXP_EXTRACT(dc_identifier,r'([0-9]{4}-[0-9A-Z]{4})\|') AS issn,
  FROM `insyspo.publicdb_redalyc_2024_02.works_pdf`
)

Query is running:   0%|          |

""


# Join tables

In [ ]:
%%bigquery --project=insyspo

DROP TABLE IF EXISTS `insyspo.publicdb_redalyc_2024_02.match_redalyc`;
CREATE TABLE `insyspo.publicdb_redalyc_2024_02.match_redalyc` AS (
  SELECT *
  FROM `insyspo.publicdb_redalyc_2024_02.match_jats`
  UNION ALL
  SELECT *, NULL AS eissn, NULL AS doi
  FROM `insyspo.publicdb_redalyc_2024_02.match_pdfs`
);

Query is running:   0%|          |

""


# Match by DOI

In [ ]:
%%bigquery --project=insyspo

SELECT b.id AS oa_id, a.id AS redalyc_id
FROM `insyspo.publicdb_redalyc_2024_02.match_redalyc` AS a
JOIN `insyspo.publicdb_openalex_2024_02_rm.works` AS b
ON LOWER(a.doi) LIKE CONCAT('%',LOWER(SUBSTR(b.doi,17)),'%')

# Sequential matching

- DOI matching - remove from the set
- Filter ISSN - find titles
- Filter Journal title  - find titles
- Select columns - cleaned title, first author, year - find match